In [6]:
import os

os.environ['NVIDIA_API_KEY'] = "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"
from omegaconf import OmegaConf
import sys
sys.path.append("../")
sys.path.append("../../../")
from retriever_evalset_generator import RetrieverEvalSetGenerator
from filters import EasinessFilter, AnswerabilityFilter
from nemo_curator.modules.filter import ScoreFilter, Score
from nemo_curator.datasets import DocumentDataset
import warnings
warnings.filterwarnings('ignore')

import dask.dataframe as dd
from distributed import Client
from distributed.diagnostics.progressbar import progress
from tqdm import tqdm


AttributeError: type object 'tqdm' has no attribute 'notebook'

In [2]:
import pandas as pd
df = pd.read_json("../data/nq_test.jsonl", lines=True)
df = df[:5]

In [3]:
cfg = OmegaConf.load("../config/retriever_eval_sdg.yaml")
retrieval_evalset_generator = RetrieverEvalSetGenerator(cfg)

In [4]:

dataset = DocumentDataset.from_pandas(df)
generated_dataset = retrieval_evalset_generator(dataset)


AttributeError: 'Series' object has no attribute 'progress_apply'

In [5]:
generated_dataset.df.head(n=5, npartitions=-1)


,_id,text,title,question-id,question,answer
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),9af84d8e5d9541d2206929e119c706e2cd3b6bd41563f6...,Who are the executive producer and producers o...,The executive producer of the fourth season of...
1,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),bfcb603517dae0edd336b86bcfa108ce4b2ac6ae2839c2...,When did the fourth season of Chicago Fire pre...,The fourth season of Chicago Fire premiered on...
2,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),e7286262331b9208ba55882dfee4e2e3d6bd7ec8714df6...,How many episodes were there in the fourth sea...,There were 23 episodes in the fourth season of...
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,9fb54dc86ca84bec822b8b78d1760cf79de754cc71ce35...,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C..."
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,c6f3fd3f4854e4d9110b25d9c5001b6635568c76bfca7a...,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ..."


In [6]:
ef = EasinessFilter(cfg)
easiness_filter = ScoreFilter(ef,
                              text_field = ["text", "question"],
                              score_field = "easiness_scores")
af = AnswerabilityFilter(cfg)
answerability_filter = ScoreFilter(af,
                              text_field = ["text", "question"],
                              score_field = "answerability_scores")

In [7]:
filtered_dataset = easiness_filter(generated_dataset)
filtered_dataset.df.head(n=5, npartitions=-1)

,_id,text,title,question-id,question,answer,easiness_scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),1960bfa176c86a428b3ef1126519109a0824c49aba2906...,Who are the executive producer and producers o...,The executive producer of the fourth season of...,0.621422
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,3462486884345c2f9bd139c3c2486aee87a3d83a7e2b1d...,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",0.583219
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,b1f5c123ee0cf1ef88d6a1c96de5c3f739f6dc67836bc7...,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",0.632935
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,6554e6aaf100c1b0df18b22fe0972a8cbde525a78e380a...,"Who wrote the song ""Fishin' in the Dark""?","""Fishin' in the Dark"" was written by Wendy Wal...",0.645659
8,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,e1098ac9773736e629b99a8566fb4ac1223276f7c86faa...,"When was ""Fishin' in the Dark"" released as a s...","""Fishin' in the Dark"" was released in June 1987.",0.636784


In [8]:
filtered_dataset_2 = answerability_filter(filtered_dataset)
filtered_dataset_2.df.head(n=5, npartitions=-1)

,_id,text,title,question-id,question,answer,easiness_scores,answerability_scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),1a06a0f200f9ee4cf525b70312b46094b8b7884c1dc3b1...,Who are the executive producer and producers o...,The executive producer of the fourth season of...,0.621422,"{\n""criterion_1_explanation"": ""The question is..."
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,d91f2033fb1d0dc74a4ee80f855b2cbb45c357e9d610b0...,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",0.583219,"{\n""criterion_1_explanation"": ""The question is..."
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,55436ae0abebbbb98092087995dcc88c7b066e75e2aea2...,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",0.632935,"{\n""criterion_1_explanation"": ""The question is..."
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,4ee4a9af666acfefc6caa6c5a4a64ef72164c167325a06...,"Who wrote the song ""Fishin' in the Dark""?","""Fishin' in the Dark"" was written by Wendy Wal...",0.645659,"{\n""criterion_1_explanation"": ""The question is..."
8,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,90d59cb2daf680e75ba2e1ff9fe22a6865b50a2f52b855...,"What was the highest chart position for ""Fishi...","""Fishin' in the Dark"" reached number-one on th...",0.597869,"{\n""criterion_1_explanation"": ""The question is..."
